# Introduction to Artificial Intelligence
Laboratory 1



Exercise 1 Route searching 
1.	Create a set of cities (as points) with coordinates x, y on a plane with height as z coordinate. The cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. You should define scenarios according to two criteria: 
a.	There are all the direct connections / c.a. 80% of possible connections
b.	The problem is symmetrical / asymmetrical (in asymmetrical – going up is height +10%, going down: -10%)
You should choose the coordinates randomly from the range <-100, 100> for x,y and <0, 50> for z.
2.	Represent the created map as a weighted (directed) graph, where cities are the nodes and roads are the edges of the graph.
3.	In the created scene, solve the traveling salesman problem: The salesman starts from a chosen city and has to visit every city exactly once before returning to the starting city. The goal is to find a path with the lowest cost.
In the problem, we define state as a partial or full path from the starting city and the corresponding state. You should represent the search problem in a form of state tree.
a.	Implement a full search of the tree, using BFS and DFS methods.
b.	Approximate the solution using greedy search (NN and Dijkstra)
c.	Solve/approximate the solution using A* with inadmissible/admissible heuristics
d.	Approximate the solution using ACO algorithm
4.	Test each algorithm, in each scenario, for n=5…20 cities, in terms of the found path cost, time and memory consumption.


In [1]:
import random
from support import euclidean_distance,euclidean_distance_assymetrical
class CitiesMap():
    def __init__(self,n,seed = None):
        """
        initialize with number of cities to include in graph 
        """    
        
        if seed is not None:
            random.seed(seed)
        self.seed = seed
        self.n = n 
        self.cities = []
        self.routes = [[0 for _ in range(n)] for _ in range(n)]
        # we can represent routes between cities as a matrix with n x n shape 
        for i in range(n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))

        # now let's make sure that we don't have duplicated cities so we'll get derired number of unique cities
        
        len_check = len(set(self.cities))
        
        while (len_check != n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))
            len_check = len(set(self.cities))
        self.cities = list(set(self.cities))
    
    
    def calculate_distance(self,symmetrical: bool = True , connections: float = 1):
        """
        Fill matrix with distnaces between city i and j  0 < i,j < n with 0 on main diagonal , conenctions 1 means all cities will be connected
        0.5 only half of them and so on 
        
        """
             
        if (symmetrical):
            if (connections==1):   
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
                return self.routes
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n)/2 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))

                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        indexes_list.append((i,j))
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
            
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes when let's say i means position with 0 index in matrix rows, j column
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0        # deletes j,i 
                    
                    
                
                
                return self.routes
                
                
                
        else: 
            if (connections==1):  
                for i in range(self.n):
                    for j in range(self.n):
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                return self.routes  
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n) # assymetrical 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))
                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(self.n):
                        # however with connections to delete we'll keep approach from previous one as I understand that lack connections means no road between two points 
                        # no matter how long 
                        
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                        # adds to indexes base only ones from above main diagonal 
                        if (j>i):
                            indexes_list.append((i,j))
                    
                    
                        
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes 
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0
                    
                    
                
                
                return self.routes
                
                
    
    


In [ ]:
class RouteSearch:
    def __init__(self,cities_map : CitiesMap):
        self.nodes = cities_map.cities
        self.size = cities_map.n
        self.edges = cities_map.routes
        # self.states = []
        from sys import maxsize
        self.max = maxsize
    def dfs_all_paths(self, starting_node: int):
        """
        Perform DFS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of paths. Path can be incomplete or complete.
        """
        # Stack for DFS
        stack = [[starting_node]]
        all_paths = []

        while stack:
            path = stack.pop()
            last_node = path[-1]
            
            
            # Record the current (incomplete or complete) path and its cost
            all_paths.append(path)

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]
                if return_to_start_cost != 0:     # to make sure that we can return to start from there
                    # Record the completed cycle (path returns to the start)
                    all_paths.append(path + [starting_node])

            # Explore all unvisited neighbors (continue DFS)
            for neighbor in range(self.size - 1, -1, -1):  # reverse order to simulate stack behavior
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    stack.append(new_path)
                    
        return all_paths    
    
    def bfs_all_paths(self, starting_node: int):
        """
        Perform BS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of paths. Path can be incomplete or complete.
        """
        # queue for bfs
        queue = [[starting_node]]
        all_paths = []

        while queue:
            path = queue.pop(0)   # take first element from list (queue)
            last_node = path[-1]
            
            # Record the current (incomplete or complete) path and its cost
            all_paths.append(path)

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]   # to make sure that we can return to start from there
                if return_to_start_cost != 0:
                    # Record the completed cycle (path returns to the start)
                    all_paths.append(path + [starting_node])

            # Explore all unvisited neighbors (continue bfs)
            for neighbor in range(0, self.size):  # normal queue order
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    queue.append(new_path)
                   

        return all_paths
    
    def get_distance(self,cycle):
        distance = 0
        for index in range(self.size):  # it's in fact len(state_tree) - 1 -> exactly what we need in this approach
            distance += self.edges[cycle[index]][cycle[index+1]]  # we need to extract ith and ith+1 element of our cycle 
        return distance
    
    def get_min_distance(self,state_tree):
        """ Requires state tree object from for example dfs or bfs algo, calculate distance only for full paths"""
        
        min_distance = self.max
        shortest_route = []
        for cycle in state_tree:
            if (len(cycle)==self.size + 1):  # we calculate distance only in case of full cycle
                distance = self.get_distance(cycle)
            
                if (distance < min_distance):
                    min_distance = distance
                    shortest_route = cycle

        return (shortest_route,min_distance)
    
    def nn(self,start):
        from sys import maxsize
        path = [start]
        
        while(len(path)!=self.size):
            dist = maxsize  # initialize distance as maxsize
            node = path[-1]  # get last node from path as a new 'starting node'
            target = -1  # initialize target as a number from outside matrix index
            for neighbour in range(self.size):   # iterate over neighbours of 'new' target                 
                cur_dist = self.edges[node][neighbour] 
                if ((neighbour not in path) & (0 < cur_dist < dist)):    # if path exists and is shorter than previous lower from this starting point 
                    target = neighbour   # store neighbour
                    dist = cur_dist   # store shortest distance 
                    
            if target == -1:    # we got in the dead end 
                return path , "Didn't find complete route"
            path.append(target)
        
        if (self.edges[target][start]!=0):    # if we reached there and we can go back to start we found complete circle
            path.append(start)
            return path , self.get_distance(path)
        
        # póki co nie testuję 
        return path, "Didn't find complete route - no connection between last node and starting node"
            
    def dijkstra(self,start):
        # mozemy wygenerować wszystkie możliwe dwuelementowe pary i wykorzystując dijkstre wygenerować ściezki dla każdej takie pary 
        # nastepnie już przechodząc przez graf możemy próbowac podążać ścieżką daje nam najmniejszy koszt/czas
        
        
        
        pass
    

In [ ]:
# dijkstra testy 
from queue import PriorityQueue

queue = PriorityQueue() # (cost, Edge(path_length, vertex_nr))
queue.put((0, Edge([start], self.get_neighbors(start)))) # najpierw path to po prostu startowy wierzchołęk
current = queue.get() # wyjmujemy z kolejki


In [32]:
# test = CitiesMap(20,seed=42)  # this will generate route without connection with nn
test = CitiesMap(11,seed=40)
test.calculate_distance(symmetrical=False,connections=1);

In [26]:
algos = RouteSearch(test)

In [ ]:

paths = algos.dfs_all_paths(0)


In [34]:
paths1 = algos.bfs_all_paths(0)

In [16]:
test.routes[5]

[78.39642849007855,
 78.82360052674579,
 84.29116205154607,
 63.85460046073423,
 182.28069014572003,
 0.0,
 162.22284056198745,
 164.93407774016868,
 114.08575721798056,
 133.9776100697426]

In [17]:
algos.edges

[[0.0,
  107.98870311287195,
  137.82234942127494,
  55.96436366117281,
  139.4964157245626,
  76.06576102294646,
  133.9897384130591,
  90.16789894413644,
  84.29211113740122,
  141.088624630053],
 [107.922008876781,
  0.0,
  44.15608678313784,
  112.20913510048992,
  157.52526781440494,
  75.46363362574056,
  114.03074146913191,
  192.2571455109016,
  77.86809359423152,
  68.33125200082317],
 [139.12224840046252,
  49.68057970676268,
  0.0,
  138.07392947258364,
  183.06624484049482,
  84.29116205154607,
  144.73164823216794,
  224.70480635714048,
  116.24353745477639,
  86.83893136145792],
 [55.67414121475068,
  112.19309247899356,
  135.85289838645326,
  0.0,
  192.123814244877,
  58.89830218266058,
  176.4328767548724,
  130.99221350904793,
  123.4514074443868,
  166.63015933497752],
 [139.81863252084824,
  158.0841864324196,
  182.82026692902514,
  192.72249479497717,
  0.0,
  182.03365073524182,
  62.64982043070834,
  146.46842663181715,
  87.42202239710541,
  118.79920033400899

In [37]:
algos.get_min_distance(paths)

([0, 3, 5, 2, 1, 8, 6, 4, 7, 0], 615.1826135503313)

In [36]:
algos.get_min_distance(paths1)

([0, 3, 5, 2, 1, 8, 6, 4, 7, 0], 615.1826135503313)

In [38]:
algos.nn(0)

([0, 3, 5, 1, 2, 8, 6, 7, 4, 0], 670.0831848922456)